In [1]:
pip install cimcb

In [2]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import cimcb as cb

from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


All packages successfully loaded


In [3]:
seed_split = 40
# seed_split = None

In [4]:
from google.colab import files

uploaded = files.upload()

Saving Data and Peak sheet_Test.xlsx to Data and Peak sheet_Test (2).xlsx


In [5]:
# The path to the input file (Excel spreadsheet)
filename = 'Data and Peak sheet_Test.xlsx'
#filename = 'MTBLS290db.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peaks')

Loadings PeakFile: Peaks
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 418 TOTAL PEAKS: 8765
Done!


In [6]:
# Clean PeakTable and Extract PeakList
RSD = peakTable['QC_RSD']
PercMiss = peakTable['Perc_missing']
PeakTableClean = peakTable[(RSD < 20) & (PercMiss < 10)]
PeakList = PeakTableClean['Name']

print("Number of peaks remaining: {}".format(len(PeakTableClean)))

# Select Subset of Data (Class "MD" or "CG" only)
DataTable2 = dataTable[(dataTable.Class == "MD") | (dataTable.Class == "CG")]

# Extract X Data
X = DataTable2[PeakList]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = [1 if outcome == 'MD' else 0 for outcome in Outcomes]
Y = np.array(Y)

# Optional: Save Class Labels for Figure Legends
Class = DataTable2.Class

Number of peaks remaining: 2259


In [7]:
# Split Data into Train (2/3rd) and Test (1/3rd)
XTrain, XTest, YTrain, YTest, ClassTrain, ClassTest = train_test_split(X,
                                                                       Y,
                                                                       Class,
                                                                       test_size=1/3,
                                                                       stratify=Y,
                                                                       random_state=seed_split)

In [8]:
# Extract X Train Data                          
XTrainLog = np.log(XTrain)
XTrainScale = cb.utils.scale(XTrainLog, method='auto')
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)

# Extract X Test Data
XTestLog = np.log(XTest)
XTestScale = cb.utils.scale(XTestLog, method='auto')
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)

/usr/local/lib/python3.7/dist-packages/cimcb/utils/knnimpute.py:92: RuntimeWarning: divide by zero encountered in true_divide
  weights = 1 / dist[1 : k + L + 1, nan_cols[i]]
/usr/local/lib/python3.7/dist-packages/cimcb/utils/wmean.py:36: RuntimeWarning: Mean of empty slice
  m = np.nanmean(x[infs])


In [9]:
np.isnan(XTrainKnn).any()

True

In [10]:
display(YTrain)

array([0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0])

In [12]:
# Parameter Dictionary
param_dict = {'n_components': [1, 2, 3, 4, 5, 6]}



# Initialise
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,
                        X=XTrainKnn,
                        Y=YTrain,
                        param_dict=param_dict,
                        folds=5,
                        n_mc=10)

# Run and Plot
cv.run()

Number of cores set to: 2
Running ...


  7%|▋         | 4/60 [00:06<01:50,  1.97s/it]

LinAlgError: ignored

In [ ]:
np.isnan(x).any()

In [ ]:
display(x)

In [ ]:
display(YTrain)

In [ ]:
print(len(y))

In [ ]:
cv.plot(metric='r2q2',
        method='absolute',
        ci=95,
        legend=True)